In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
from ase.io import read

In [2]:
# ended up not using

def getEnergy(outPath:str):
    '''Get calculated energy from molpro.out'''
    with open(outPath, "r") as file: 
        for line in file:
            if 'DF-KS/aug-cc-pVTZ energy=' in line:
                return float(line.split('   ')[1])

In [3]:
# test
getEnergy('/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Benzene/DF-KS_LDA/aug-cc-pVTZ/Benzene-Benzene_pi-pi_0.90/molpro.out')

-460.364314311695

In [4]:
molpro_folder='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Benzene'

files=[]
for (dirpath, dirnames, filenames) in os.walk(molpro_folder):
    for filename in filenames:
        files.append(f'{dirpath}/{filename}')

files=list(filter(lambda x:'ase.xyz' in x, files))

data_molpro=pd.DataFrame({
    'Calculation type': 'Ab initio',
    'Method': '',
    'Distance': [0]*len(files),
    'Energy': 0
})

In [5]:
for i, row in data_molpro.iterrows():
    data_molpro.loc[i, 'Method'] = files[i].split('/')[-4]
    data_molpro.loc[i, 'Distance'] = files[i].split('/')[-2].split('_')[-1]

    atoms=read(files[i],format='extxyz')
    data_molpro.loc[i, 'Energy'] = atoms.get_total_energy()

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_57142/1374398513.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.90' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_molpro.loc[i, 'Distance'] = files[i].split('/')[-2].split('_')[-1]
/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_57142/1374398513.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-12627.129353934697' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_molpro.loc[i, 'Energy'] = atoms.get_total_energy()


In [19]:
data_mace=pd.read_pickle('/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Benzene/MACE/results.pkl')
data_mace['Method']='MACE'+'-'+data_mace['Model']

df=pd.concat([data_molpro,data_mace], ignore_index=True)

In [20]:
# calculate interaction energy (total energy minus monomers)
df['Interaction energy [eV]']=0

for method in df['Method'].unique():
    subset=df.query(f'Method == "{method}"')
    df.loc[df['Method']==method, 'Interaction energy [eV]'] = subset['Energy'] - float(subset.loc[subset['Distance']=='1', 'Energy']) - float(subset.loc[subset['Distance']=='2', 'Energy'])

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_57142/4006019392.py:6: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_57142/4006019392.py:6: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 3.63122542e-02 -1.11152012e-01  6.31358283e+03 -8.32900603e-02
 -6.53206351e-02 -2.93166973e-02 -1.25251575e-01 -4.07935387e-03
  6.31358284e+03 -1.22984097e-01]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_57142/4006019392.py:6: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/var/folders/22/kgbcd6856bb9_kzpfp_36k600000gn/T/ipykernel_57142/4006019392.py:6:

In [21]:
df=df[df['Distance'].str.len()>=2] # drop full integer values (Distance 2 & 1 represent individual benzene and not dimer)
df['Distance']=df['Distance'].astype(float)
df=df.sort_values('Distance').reset_index(drop=True)

In [24]:
plot=px.line(df, x='Distance', y='Interaction energy [eV]', color='Method', markers=True, template='ggplot2')

In [25]:
plot.show()